# Introduction

- Open-source development framework for LLM applications
- Python and JavaScript (TypeScript) packages
- Focused on compostion and modularity

Key value adds:
1. Modular components
2. Use cases: Common ways to combine components

## Components of LangChain

### Models
- LLMs: 20+ integrations
- Chat Models
- Text Embedding Models: 10+ integrations

### Prompts
- Prompt Templates
- Output Parsers: 5+ implementations
    - Retry/fixing logic
- Example Selectors: 5+ implementations

### Indexes
- Document Loaders: 50+ implementation
- Text Splitters: 10+ implementation
- Vector Stores: 10+ integrations
- Retrievers: 5+ integrations/implementations

### Chains
- Prompt + LLM + Output parsing
- Can be used as building blocks for longer chains
- More application specific chains: 20+ types

### Agents
- Agent Types: 5+ types
    - Algorithms for getting LLMs to use tools
- Agent Toolkits: 10+ implementation
    - Agents armed with specific tools for a specific application

# Models, Prompts and Parsers

In [145]:
import os
os.environ["NVIDIA_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

## Direct API Calls

### NVIDIA Implementation
Let's start with direct API calls to NVIDIA ~~OpenAI~~

In [2]:
import requests, base64, os

def get_completion(prompt):
    invoke_url = "https://ai.api.nvidia.com/v1/vlm/microsoft/phi-3-vision-128k-instruct"
    
    headers = {
      "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
      "Accept": "application/json"
    }
    
    payload = {
      "messages": [
        {
          "role": "user",
          "content": f'{prompt}'
        }
      ],
      "max_tokens": 512,
      "temperature": 1.00,
      "top_p": 0.70
    }
    response = requests.post(invoke_url, headers=headers, json=payload)
    return response.json()['choices'][0]['message']['content']


# response = get_completion("What is 1+1?").json()
# print(response['choices'][0]['message']['content'])
response = get_completion("What is 1+1?")
print(response)

1+1 is equal to 2. This is a basic arithmetic operation known as addition, where 1 is added to another 1 to produce a sum of 2.


### Ollama Implementation

In [3]:
invoke_url = "http://localhost:11434/api/generate"

def get_completion_local(prompt):
    headers = {
        "Accept": "application/json"
    }
    
    payload = {
      "model": "llama3",
      "prompt":f"{prompt}",
      "stream": False
    }
    
    response = requests.post(invoke_url, json=payload)
    return response.json()["response"]

get_completion_local("What is 1+1?")

"That's an easy one!\n\nThe answer is: 2!"

### OpenAI Implementation: Just in Case

In [4]:
#!pip install openai

In [5]:
# import openai

# os.environ["NVIDIA_API_KEY"] = XXXXXXXXXXXXXX
# openai.api_key = os.environ['OPENAI_API_KEY']
# llm_model = "gpt-3.5-turbo"

# def get_completion(prompt, model=llm_model):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0, 
#     )
#     return response.choices[0].message["content"]

# get_completion("What is 1+1?")

### Langchain Abstractions

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone

text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
print(get_completion(prompt))

Oh, I am quite frustrated that my blender lid came off and caused a mess in my kitchen. And to add insult to injury, the warranty does not cover the cost of cleaning up my kitchen. I need your help right away, matey!


In [8]:
print(get_completion_local(prompt))

Here is the translated text in a calm and respectful tone using American English:

```
Gosh, I'm really upset that my blender lid came loose and splattered smoothie all over my kitchen walls! And to make things worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now.
```

Let me know if you'd like any further adjustments!


## ChatAPI LangChain

### Why Use Prompt Templates?

![Why Use Prompt Templates](data/images/why-use-prompt-templates.png)

![LangChain output parsing works with prompt templates](data/images/output-parsing-works-with-prompts.png)

In [9]:
# !pip install --upgrade langchain
# !pip install langchain-nvidia-ai-endpoints

In [10]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [12]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [13]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [14]:
customer_style = """American English \
in a calm and respectful tone
"""

In [15]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [16]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [17]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [18]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [19]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [20]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



### ChatNVIDIA

In [309]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

llm_nvidia = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

In [22]:
customer_response = llm_nvidia.invoke(customer_messages)

In [23]:
print(customer_response.content)

 I'm really upset that the lid of my blender came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I need your help right now, friend!


In [24]:
service_response = llm_nvidia(service_messages)
print(service_response.content)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 Ahoy there, me hearty customer! I'm afraid I've got some unfortunate news for ye. The warranty on yer kitchen appliance, specifically the blender, does not cover the costs of cleanin' up after a mishap. It seems ye may have forgotten to put the lid on before startin' the blender, which led to the mess. I'm truly sorry, but them's the rules, I'm afraid. We can't be coverin' every little mishap, as much as we'd like to help. I hope ye understand and have a smoother sailin' in the future. Fare thee well, me friend!


### ChatOllama

In [25]:
from langchain_community.chat_models import ChatOllama

llm_ollama = ChatOllama(model="llama3")

In [26]:
customer_response = llm_ollama(customer_messages)
print(customer_response.content)

Here's the translation:

```
Ugh, I'm absolutely livid that my blender lid flew off and splattered smoothie all over my kitchen walls! To make things worse, the warranty doesn't cover the cost of cleaning up the mess. I really need some help right now - can you lend a hand? 
```

I've kept the tone calm and respectful, while still conveying your frustration with the situation. I've also used American English spellings (e.g., "absolutely" instead of "be", "livid" instead of "fuming", etc.) to ensure the translation is suitable for an American audience.


In [27]:
service_response = llm_ollama(service_messages)
print(service_response.content)

Arrrr, greetings me hearty customer!

I be afraid that yer warranty don't cover them fancy cleanin' expenses fer yer galley (that's kitchen speak, matey). Seemingly, ye got yerself into a spot o' trouble by forgettin' to put the lid on yer trusty blender afore startin' it up. Shiver me timbers! A bit o' common sense be in order here, don't ye think?

Well, matey, it seems like ye've got yerself a case o' "Tough Luck" comin' atcha! But never fear, there be more where that came from! Just keep in mind that next time ye set sail with yer blender, make sure to have the lid securely fastened, or ye might just find yerself walkin' the plank!

Fair winds and following seas, me hearty! May yer kitchen be as smooth as a calm sea on a sunny day!


### ChatOpenAI

In [28]:
# from langchain.chat_models import ChatOpenAI
# llm_openai = ChatOpenAI(temperature=0.0, model=llm_model)

## Output Parsers

How we would like the LLM output to look like:

In [29]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [30]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [31]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [32]:
messages = prompt_template.format_messages(text=customer_review)

In [33]:
response = llm_nvidia(messages)
print(response.content)

 {
"gift": true,
"delivery_days": 2,
"price\_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [34]:
response = llm_ollama(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
    "gift": True,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Note: The `price_value` key has a list value because there is only one sentence that provides information about the price or value of the product.


In [35]:
# response = llm_openai(messages)

In [36]:
type(response.content)

str

#### Parse the LLM output string into a Python dictionary

In [37]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [38]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer 'True' if yes,\
                             'False' if not or unknown. Answer should be within quotes")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [39]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [40]:
format_instructions = output_parser.get_format_instructions()

In [41]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer 'True' if yes,                             'False' if not or unknown. Answer should be within quotes
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [42]:
review_template_2 = """\
For the following text, extract the following information. 
Don't respond to me back.
Just give me the information.

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [43]:
print(messages)

[HumanMessage(content='For the following text, extract the following information. \nDon\'t respond to me back.\nJust give me the information.\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown co

In [44]:
response = llm_nvidia(messages)
print(response.content)

 ```json
{
	"gift": "False",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [45]:
output_dict = output_parser.parse(response.content)
print(output_dict)

{'gift': 'False', 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}


In [46]:
response = llm_ollama(messages)
print(response.content)

```json
{
    "gift": "True",
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [47]:
output_dict = output_parser.parse(response.content)

In [48]:
output_dict['price_value']

["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]

# Memory

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

![LangChain Memory](data/images/langchain-memory.png)

### ConversationBufferMemory

In [49]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [50]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm_nvidia,
    memory = memory,
    verbose = True
)

In [51]:
conversation.predict(input="Hi my name is Subash")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi my name is Subash
AI:

> Finished chain.


" Hello Subash, it's nice to meet you! I'm an AI and I'm here to help answer your questions and engage in conversation. How has your day been so far? If you have any questions or topics you'd like to discuss, feel free to let me know!\n\n(Note: As a helpful and talkative AI, I'll provide detailed responses and context where appropriate, and if I don't know an answer, I'll let you know honestly.)"

In [52]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Subash
AI:  Hello Subash, it's nice to meet you! I'm an AI and I'm here to help answer your questions and engage in conversation. How has your day been so far? If you have any questions or topics you'd like to discuss, feel free to let me know!

(Note: As a helpful and talkative AI, I'll provide detailed responses and context where appropriate, and if I don't know an answer, I'll let you know honestly.)
Human: What is 1+1?
AI:

> Finished chain.


" The sum of 1 + 1 is 2. This is a basic arithmetic operation that falls under the field of mathematics. Is there anything else you would like to know about mathematics, or would you like to discuss a different topic? I'm here to help and engage in conversation, so feel free to ask me anything!"

In [53]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Subash
AI:  Hello Subash, it's nice to meet you! I'm an AI and I'm here to help answer your questions and engage in conversation. How has your day been so far? If you have any questions or topics you'd like to discuss, feel free to let me know!

(Note: As a helpful and talkative AI, I'll provide detailed responses and context where appropriate, and if I don't know an answer, I'll let you know honestly.)
Human: What is 1+1?
AI:  The sum of 1 + 1 is 2. This is a basic arithmetic operation that falls under the field of mathematics. Is there anything else you would like to know about mathematics, or would you like to discuss a different topic? I'm here

' Your name, as you previously told me, is Subash. Is there anything you would like to know or discuss related to your name, or would you like to talk about something else? I strive to provide detailed and helpful responses, so please let me know how I can assist you further!'

In [54]:
print(memory.buffer)

Human: Hi my name is Subash
AI:  Hello Subash, it's nice to meet you! I'm an AI and I'm here to help answer your questions and engage in conversation. How has your day been so far? If you have any questions or topics you'd like to discuss, feel free to let me know!

(Note: As a helpful and talkative AI, I'll provide detailed responses and context where appropriate, and if I don't know an answer, I'll let you know honestly.)
Human: What is 1+1?
AI:  The sum of 1 + 1 is 2. This is a basic arithmetic operation that falls under the field of mathematics. Is there anything else you would like to know about mathematics, or would you like to discuss a different topic? I'm here to help and engage in conversation, so feel free to ask me anything!
Human: What is my name?
AI:  Your name, as you previously told me, is Subash. Is there anything you would like to know or discuss related to your name, or would you like to talk about something else? I strive to provide detailed and helpful responses, s

In [55]:
memory.load_memory_variables({})

{'history': "Human: Hi my name is Subash\nAI:  Hello Subash, it's nice to meet you! I'm an AI and I'm here to help answer your questions and engage in conversation. How has your day been so far? If you have any questions or topics you'd like to discuss, feel free to let me know!\n\n(Note: As a helpful and talkative AI, I'll provide detailed responses and context where appropriate, and if I don't know an answer, I'll let you know honestly.)\nHuman: What is 1+1?\nAI:  The sum of 1 + 1 is 2. This is a basic arithmetic operation that falls under the field of mathematics. Is there anything else you would like to know about mathematics, or would you like to discuss a different topic? I'm here to help and engage in conversation, so feel free to ask me anything!\nHuman: What is my name?\nAI:  Your name, as you previously told me, is Subash. Is there anything you would like to know or discuss related to your name, or would you like to talk about something else? I strive to provide detailed and 

In [56]:
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, {"output": "What's up"})

In [57]:
print(memory.buffer)

Human: Hi
AI: What's up


In [58]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [59]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [60]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### ConversationBufferWindowMemory

In [61]:
from langchain.memory import ConversationBufferWindowMemory

In [62]:
memory = ConversationBufferWindowMemory(k=1)

In [63]:
memory.save_context({"input":"Hi"},{"output":"What's up"})
memory.save_context({"input":"Not much, just hanging"}, {"output":"Cool"})

In [64]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [65]:
memory = None

It only remembers only latest conversation because k = 1

In [66]:
memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(
    llm = llm_ollama,
    memory = memory,
    verbose = True
)

In [67]:
conversation.predict(input="Hi, I'm Subash")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Subash
AI:

> Finished chain.


"Nice to meet you, Subash! I'm LLaMA, an AI designed by Meta AI that can converse in a human-like way. I've been trained on a massive dataset of text from the internet and can provide information on a wide range of topics. Currently, my knowledge cutoff is up until 2022, so if there's anything more recent than that, I might not be aware of it. But don't worry, I'll do my best to help answer your questions or engage in a fun conversation with you!"

In [68]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Subash
AI: Nice to meet you, Subash! I'm LLaMA, an AI designed by Meta AI that can converse in a human-like way. I've been trained on a massive dataset of text from the internet and can provide information on a wide range of topics. Currently, my knowledge cutoff is up until 2022, so if there's anything more recent than that, I might not be aware of it. But don't worry, I'll do my best to help answer your questions or engage in a fun conversation with you!
Human: What is 1+1?
AI:

> Finished chain.


"Easy one, Subash! The answer to 1+1 is... (drumroll please)... 2! Yes, the result of adding one and one together is indeed two. This fundamental arithmetic operation is a building block for more complex math concepts, and it's a basic fact that I've been trained on through my vast dataset of text from the internet."

In [69]:
conversation.predict(input="Who am I?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: Easy one, Subash! The answer to 1+1 is... (drumroll please)... 2! Yes, the result of adding one and one together is indeed two. This fundamental arithmetic operation is a building block for more complex math concepts, and it's a basic fact that I've been trained on through my vast dataset of text from the internet.
Human: Who am I?
AI:

> Finished chain.


"A question about identity! Well, Subash, as far as our conversation goes, you are... (drumroll please)... the human who is chatting with me, an AI designed to simulate human-like conversations! You're a curious individual who's asking thoughtful questions and engaging in a friendly chat with me. I don't have any specific information about your real-life identity or personal details, but based on our conversation, you seem like a bright and inquisitive person!"

### ConversationTokenBufferMemory

Memory will limit the number of tokens. Since, a lot of LLM pricing is based on tokens, this maps more directly to the cost of LLM calls.

Different llms use different ways of counting tokens. Passing llm tells the ConversationTokenBufferMemory to use the way of counting token that the llm passed uses.

In [70]:
from langchain.memory import ConversationTokenBufferMemory

In [71]:
memory = ConversationTokenBufferMemory(llm=llm_nvidia, max_token_limit=25)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

/home/mojo/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [72]:
memory.load_memory_variables({})

{'history': 'Human: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### ConversationSummaryBufferMemory

In [73]:
from langchain.memory import ConversationSummaryBufferMemory

In [74]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm_nvidia, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [75]:
print(memory.load_memory_variables({}))

{'history': 'System:  The human and AI exchange greetings, with the human mentioning they are just hanging out. The human then asks about the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}


In [76]:
conversation = ConversationChain(
    llm = llm_nvidia,
    memory = memory,
    verbose = True
)

In [77]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System:  The human and AI exchange greetings, with the human mentioning they are just hanging out. The human then asks about the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


" I would recommend showing the latest Language Model (LLM) demo that we have been working on. It incorporates the latest advancements in natural language processing and can effectively demonstrate the capabilities of our AI technology. This demo involves real-time language translation, text summarization, and sentiment analysis. The customer you're meeting with will be particularly interested in the translation feature, as they are coming from a different region. The demo will help illustrate how our AI can assist in overcoming language barriers and facilitate smoother communication."

![Memory Types](data/images/memory-types.png)

![Additional Memory Types](data/images/additional-memory-types.png)

# Chains

The chain usually combines a LLM together with a prompt and put a bunch of building blocks together to carry out a sequence of operations on your text/data.

In [78]:
import pandas as pd

product_review = {
    "product": ["Queen Size Sheet Set",
                "Waterproof Phone Pouch",
                "Luxury Air Mattress",
                "Pillows Insert",
                "Milk Frother Handheld",
                "L'Or Espresso Café",
                "Hervidor de Agua Eléctrico",
               ],
    "review":[
        "I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.",
        "I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.",
        "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
        "This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great",
        "I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
        "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
        "Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento."        
    ]
}


In [79]:
df = pd.DataFrame(product_review)

In [80]:
df.head()

,product,review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...


## LLMChain

In [81]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [82]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give me only one name"
)

In [83]:
chain = LLMChain(llm=llm_nvidia, prompt=prompt)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [84]:
product = "Queen Size Sheet Set"
chain.run(product)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' "Monarch Regalia Linens" could be a fitting name for a company that makes queen size sheet sets, as it evokes a sense of luxury and grandeur, fitting for a queen-sized bed. The name also includes "linens" to clearly communicate the nature of the company\'s products.'

## Sequential Chains

![Sequential Chain](data/images/sequential-chain.png)

### Simple Sequential Chain

![Simple Sequential Chain](data/images/simple-sequential-chain.png)

In [85]:
from langchain.chains import SimpleSequentialChain

In [86]:
first_prompt = ChatPromptTemplate.from_template(
    "Give me only the best name to describe the company that makes {product}."
)

chain_one = LLMChain(llm=llm_nvidia, prompt=first_prompt)

In [87]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company {company}"
)

chain_two = LLMChain(llm=llm_nvidia, prompt=second_prompt)

In [88]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], 
    verbose=True
)

In [89]:
product = "Queen Size Sheet"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
 "Royal Comfort: The Premier Provider of Queen Size Sheets"

This name effectively communicates the company's focus on creating high-quality queen size sheets, while also conveying a sense of luxury and elegance. The use of "Royal" and "Premier" highlights the brand's commitment to providing top-notch products, while "Comfort" emphasizes the softness and coziness of their sheets. Overall, this name is memorable, evocative, and clearly communicates the company's mission and values.
 "Royal Comfort: Luxurious Queen-Size Sheets for Ultimate Comfort."

> Finished chain.


' "Royal Comfort: Luxurious Queen-Size Sheets for Ultimate Comfort."'

Simple Sequential Chain works well when there is a single input and single output.

### SequentialChain

![Sequential Chain](data/images/sequential-chain-figure.png)

In [90]:
from langchain.chains import SequentialChain

In [91]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english: "
    "{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm_nvidia, prompt=first_prompt, output_key="English_Review")

In [92]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm_nvidia, prompt=second_prompt, 
                     output_key="summary"
                    )

In [93]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm_nvidia, 
                       prompt=third_prompt,
                       output_key="language"
                      )

In [94]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "Summary: {summary}"
    "Language: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm_nvidia, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [95]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [96]:
review = df.review[5]
overall_chain(review)



> Entering new SequentialChain chain...


/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
 'English_Review': " I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in the store and the taste is much better... Old lot or counterfeit!?\n\nNote: The review is expressing dissatisfaction with the taste and consistency of a product, and raises the possibility that it may be an old batch or a counterfeit version.",
 'summary': ' The reviewer finds the taste of the product mediocre and its foam inconsistent, suspecting it may be an old or counterfeit batch, and compares it unfavorably to the same product purchased in stores.',
 'followup_message': " Cher rédacteur de la revue,\n\nJe suis désolé d'entendre que votre expérience avec notre produit n'a pas été à la hauteur de vos attentes. Nous prenons vos commentaires au sérieux et allons enquêter sur la possibilité d'un lot anci

## Router Chain

![Router Chain](data/images/router-chain.png)

In [97]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [98]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [99]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [100]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_nvidia, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [101]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [102]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm_nvidia, prompt=default_prompt)

In [103]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input.
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [104]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm_nvidia, router_prompt)

In [105]:
print(router_template.format(input="What is 1+1?"))

Given a raw text input to a 
language model select the model prompt best suited for the input.
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{
    "destination": name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for an

In [106]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [107]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


" Black body radiation is the electromagnetic radiation emitted by a perfect absorber, also known as a black body, at a given temperature. It is characterized by its continuous spectrum and the fact that the amount of radiation emitted is only determined by the temperature of the black body, not by its shape, size, or composition. The peak wavelength of the radiation shifts to shorter wavelengths as the temperature increases, following the Planck's law of black body radiation. This phenomenon is important in understanding the behavior of radiation in various physical systems, such as stars and heated objects."

In [108]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


' Thank you for your kind words! I\'m here to help answer your math question to the best of my ability.\n\nThe question you have asked is: what is 2 + 2.\n\nTo answer this question, I will break it down into its component parts:\n\n* The number 2\n* The addition operation\n* The number 2\n\nWhen we perform the addition operation on the two numbers, we get:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "what is 2 + 2" is 4. I hope this helps! Do you have any other math questions that I can assist you with?'

In [109]:
chain.run("Who was Alexandar Hamilton?")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was Alexandar Hamilton?'}
> Finished chain.


" Alexander Hamilton (1755-1804) was a founding father of the United States, who served as the first Secretary of the Treasury from 1789 to 1795. He was born on the island of Nevis in the British West Indies and was orphaned at a young age. Through hard work and determination, he was able to attend King's College (now Columbia University) in New York City.\n\nDuring the American Revolution, Hamilton served as an artillery captain in the Continental Army and became a close aide to General George Washington. After the war, he played a key role in the drafting and ratification of the U.S. Constitution, and was one of the authors of the Federalist Papers, a series of essays arguing for the adoption of the Constitution.\n\nAs Secretary of the Treasury, Hamilton implemented a number of policies that helped to establish the credit of the new nation, including the creation of a national bank and the assumption of state debts. He also advocated for a strong central government and a robust feder

# Question and Answer

![LLMS on Documents](data/images/llms-on-documents.png)

![Embeddings](data/images/embeddings.png)

![Vector Database](data/images/vector-database.png)

![Vector Database to LLM](data/images/vector-database-to-llm.png)

In [110]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

### VectorstoreIndexCreator

In [111]:
file = 'data/files/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [112]:
from langchain.indexes import VectorstoreIndexCreator

In [113]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

/home/mojo/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [114]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

/home/mojo/miniconda3/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [115]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [116]:
response = index.query(query, 
                       llm = llm_nvidia)

In [117]:
display(Markdown(response))

 | Product ID | Name | Description | Fabric & Care | Sun Protection |
| --- | --- | --- | --- | --- |
| 679 | Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart™ protection, UPF 50+, wrinkle-resistant, machine washable | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |
| 255 | Sun Shield Shirt by | High-performance sun shirt with UPF 50+, wicks moisture, quick-drying, abrasion-resistant, handwash and line dry | 78% nylon, 22% Lycra Xtra Life fiber | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays, recommended by The Skin Cancer Foundation as an effective UV protectant |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Lightweight, UPF 50+ hot-weather shirt, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |
| 709 | Sunrise Tee | Women's UV-protective button-down shirt, lightweight, wicks moisture, quick-drying, wrinkle-free, UPF 50+ | Lightweight performance synthetic wicks moisture, resists wrinkles and dries fast. Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |

All the shirts listed provide sun protection with a rating of UPF 50+, blocking 98% of the sun's harmful rays. The first product is a sleeveless button-up shirt for women made of nylon and polyester, with SunSmart™ protection, wrinkle-resistant, and machine washable. The second product is a sun shirt for men made of nylon and Lycra Xtra Life fiber, with moisture-wicking, quick-drying, and abrasion-resistant features, handwash and line dry. The third product is a short-sleeve shirt for men made of 100% polyester, with front and back cape venting, two front bellows pockets, and imported. The last product is a women's UV-protective button-down shirt made of lightweight performance synthetic, wicks moisture, quick-drying, and wrinkle-free.

### Step by Step

Question answering over documents consists of four steps:
1) Create an index
2) Create a Retriever from that index
3) Create a question-answering chain
4) Ask questions!


To use retrieval in LangChain, you can follow these steps:
1) Load documents: Use document loaders to load documents from various sources, such as files, websites, or databases.
1) Transform documents: Apply document transformers to preprocess and transform the loaded documents, such as splitting large documents into smaller chunks or applying specific logic optimized for different document types.
1) Create embeddings: Generate embeddings for the documents using text embedding models. Embeddings capture the semantic meaning of text and enable efficient searching and similarity calculations.
1) Store documents and embeddings: Use vector stores to store the documents and their corresponding embeddings. Vector stores provide efficient storage and retrieval capabilities for large collections of embeddings.
1) Retrieve relevant documents: Use retrievers to query the vector store and retrieve relevant documents based on user queries or search criteria. Retriever algorithms, such as similarity search or Maximum Marginal Relevance (MMR) search, can be used to find the most relevant documents.

#### Fully Manual Way of Doing Things

In [118]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [119]:
docs = loader.load()

In [120]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [121]:
embed = embedding.embed_query("Subash")

In [122]:
embed[:5]

[0.014350582845509052,
 -0.049315445125103,
 0.013720898889005184,
 -0.05654430389404297,
 0.08923717588186264]

In [123]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embedding
)

In [124]:
query = "Please suggest a shirt with sunblocking"

In [125]:
response_docs = db.similarity_search(query)

In [126]:
response_docs[0]

Document(page_content=": 709\nname: Sunrise Tee\ndescription: Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly.\n\nSize & Fit\nSlightly Fitted: Softly shapes the body. Falls at hip.\n\nWhy We Love It\nOur lightest hot-weather shirt lets you beat the heat. Originally designed for fishing, it's also a great choice for travel thanks to its wrinkle-free fabric and built-in sun protection with a rating of UPF 50+.\n\nFabric & Care\nLightweight performance synthetic wicks moisture, resists wrinkles and dries fast. Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry.\n\nAdditional Features\nBuilt-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. The high-performance fabric keeps you cool and comfortable by wicking perspiration away. Smoother buttons, low-profile pockets and side shaping for a flattering fit. Front

If we were doing this by hand, we would combine the documents into a single piece of text.

In [127]:
qdocs = "".join([docs[i].page_content for i in range(len(response_docs))])

In [128]:
response = llm_nvidia.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [129]:
display(Markdown(response))

 | Name | Description | Size & Fit | Specs | Construction | Fabric & Care | Additional Features |
| --- | --- | --- | --- | --- | --- | --- |
| Women's Campside Oxfords | Ultracomfortable lace-to-toe Oxford with soft canvas, thick cushioning, and quality construction. | Order regular shoe size. For half sizes not offered, order up to next whole size. | Approx. weight: 1 lb.1 oz. per pair. | Soft canvas material, EVA innersole with Cleansport NXT® antimicrobial odor control, EVA foam midsole for cushioning and support, chain-tread-inspired molded rubber outsole. | Not applicable | Not applicable |
| Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece | Bright colored, ruffles and whimsical prints two-piece swimsuit for toddlers. | Designed for infants and toddlers. | Not applicable | Four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric, crossover no-slip straps, fully lined bottom. | Machine wash and line dry. | Crossover no-slip straps, fully lined bottom, UPF 50+ rated fabric. |
| Refresh Swimwear, V-Neck Tankini Contrasts | Watersport-ready tankini top in eye-catching colorblock style. | Fitted: Sits close to the body. | UPF 50+ rated – the highest rated sun protection possible. | Premium Italian-blend of 82% recycled nylon with 18% Lycra® spandex, lightweight racerback straps, flattering V-neck silhouette. | Handwash, line dry. | Lightweight racerback straps, flattering V-neck silhouette, UPF 50+ rated fabric. |

There are 3 items listed with sun protection:

1. Women's Campside Oxfords: These shoes offer sun protection through their soft canvas material, which is a good barrier against the sun's harmful rays.
2. Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece: This toddler's swimsuit features UPF 50+ rated fabric, providing the highest rated sun protection possible and blocking 98% of the sun's harmful rays.
3. Refresh Swimwear, V-Neck Tankini Contrasts: This tankini top is made of UPF 50+ rated fabric, offering the highest rated sun protection possible and blocking 98% of the sun's harmful rays.

#### Using RetrievalQA

For Question Answer over our own documents, we need to create a  **Retriever** from this **Vector Store**.

**Retriever** is a generic interface that can be underpinned by any method that takes in a query and returns documents.

In [130]:
retriever = db.as_retriever()

All these steps can be encapsulated with the LangChain chain. Here we can create a retrieval QA chain. This does retrieval and does Question Answering over the retrieved documents.

![Stuff Method](data/images/stuff-method.png)

![Additional Chain Types](data/images/additional-chain-types.png)

In [131]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm_nvidia,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [132]:
query = "Please list all your shirts with sun protection in a table in markdown and summarize each one."

In [133]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [134]:
display(Markdown(response))

 | Name | Description | Size & Fit | Fabric & Care | Additional Features | Sun Protection |
| --- | --- | --- | --- | --- | --- |
| Sunrise Tee | Lightweight, high-performance shirt for hot weather with UV-protection and moisture-wicking fabric. | Slightly Fitted, falls at hip. | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry. | Built-in SunSmart™ UPF 50+ rated, wrinkle-free, front and back cape venting, two front pockets, tool tabs and eyewear loop. | Provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart™ protection and a fit to flatter. | Slightly Fitted, falls at hip. | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry. | Updated design with smoother buttons, wrinkle resistant, front and back cape venting, two front pockets, tool tabs and eyewear loop. | Provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |
| Sun Shield Shirt | High-performance sun shirt that protects from harmful UV rays, recommended by The Skin Cancer Foundation. | Slightly Fitted, falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber. Handwash, line dry. | Wicks moisture, abrasion resistant, fits comfortably over swimsuit. | Provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightweight, hot-weather shirt with UPF 50+ sun protection and wrinkle-resistant fabric. | Traditional fit, relaxed through chest, sleeve, and waist. | 100% polyester. Machine wash and dry. | Front and back cape venting, two front bellows pockets. | Provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |

In summary, all the shirts listed provide sun protection with a UPF 50+ rating, which blocks 98% of the sun's harmful rays. The Sunrise Tee and Women's Tropical Tee, Sleeveless are lightweight, high-performance shirts with moisture-wicking fabric and are suitable for hot weather. The Sun Shield Shirt is a high-performance sun shirt that is recommended by The Skin Cancer Foundation and fits comfortably over a swimsuit. The Men's Tropical Plaid Short-Sleeve Shirt is a lightweight, hot-weather shirt with front and back cape venting and two front bellows pockets.

#### Using VectorStoreIndexCreator

VectorstoreIndexCreator is just a wrapper around all this logic.

A lot of the magic is being hidden in this: VectorstoreIndexCreator.
Three main steps are going on after the documents are loaded:
1) Splitting documents into chunks
2) Creating embeddings for each document
3) Storing documents and embeddings in a vectorstore

In [135]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding,
).from_loaders([loader])
response = index.query(query, llm=llm_nvidia)

In [136]:
display(Markdown(response))

 | Product ID | Name | Description | Fabric & Care | Sun Protection |
| --- | --- | --- | --- | --- |
| 679 | Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart™ protection, UPF 50+, wrinkle-resistant, machine washable | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |
| 255 | Sun Shield Shirt by | High-performance sun shirt with UPF 50+, wicks moisture, quick-drying, abrasion-resistant, handwash and line dry | 78% nylon, 22% Lycra Xtra Life fiber | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays, recommended by The Skin Cancer Foundation as an effective UV protectant |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Lightweight, UPF 50+ hot-weather shirt, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |
| 709 | Sunrise Tee | Women's UV-protective button-down shirt, lightweight, wicks moisture, quick-drying, wrinkle-free, UPF 50+ | Lightweight performance synthetic wicks moisture, resists wrinkles and dries fast. Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | High-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays |

All the shirts listed provide sun protection with a rating of UPF 50+, blocking 98% of the sun's harmful rays. The first product is a sleeveless button-up shirt for women made of nylon and polyester, with SunSmart™ protection, wrinkle-resistant, and machine washable. The second product is a sun shirt for men made of nylon and Lycra Xtra Life fiber, with moisture-wicking, quick-drying, and abrasion-resistant features, handwash and line dry. The third product is a short-sleeve shirt for men made of 100% polyester, with front and back cape venting, two front bellows pockets, and imported. The last product is a women's UV-protective button-down shirt made of lightweight performance synthetic, wicks moisture, quick-drying, and wrinkle-free.

# LangChain Evaluation

## Create our QandA application

In [137]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [138]:
file = 'data/files/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [139]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = embedding
).from_loaders([loader])

/home/mojo/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [140]:
qa = RetrievalQA.from_chain_type(
    llm=llm_nvidia, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [141]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

### Hard-coded examples

In [184]:
examples = [
    {"query": "Do the Cozy Comfort Pullover Set\
    have side pockets?",
    "answer": "Yes"
    },
     {"query": "What collection is the Ultra-Lofty \
    850 Stretch Down Hooded Jacket from?",
    "answer": "The DownTek collection"
     }
]


### LLM-Generated examples

In [185]:
from langchain.evaluation.qa import QAGenerateChain

In [186]:
example_gen_chain = QAGenerateChain.from_llm(ChatNVIDIA())

In [187]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [188]:
new_examples = [ex.get("qa_pairs") for ex in new_examples]

In [189]:
new_examples[0]

{'query': "What is the material of the Women's Campside Oxfords and what is its special feature?",
 'answer': "The Women's Campside Oxfords are made of soft canvas material, which provides a broken-in feel and look. One special feature of these shoes is the Cleansport NXT® antimicrobial odor control in the innersole."}

In [208]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [190]:
examples += new_examples

In [191]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set    have side pockets?',
 'answer': 'Yes'}

In [193]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


' Yes, the Cozy Comfort Pullover Set does have side pockets. This information can be found in the "Additional Features" section of the set\'s description. It mentions that the pull-on pants have "side pockets."'

### Manual Evaluation¶

In [194]:
import langchain
langchain.debug = True

In [195]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set    have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set    have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants

' Yes, the Cozy Comfort Pullover Set does have side pockets. This information can be found in the "Additional Features" section of the set\'s description. It mentions that the pull-on pants have "side pockets."'

In [196]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [197]:
examples[2]

{'query': "What is the material of the Women's Campside Oxfords and what is its special feature?",
 'answer': "The Women's Campside Oxfords are made of soft canvas material, which provides a broken-in feel and look. One special feature of these shoes is the Cleansport NXT® antimicrobial odor control in the innersole."}

In [198]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [199]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set    have side pockets?',
  'answer': 'Yes',
  'result': ' Yes, the Cozy Comfort Pullover Set does have side pockets. This information can be found in the "Additional Features" section of the set\'s description. It mentions that the pull-on pants have "side pockets."'},
 {'query': 'What collection is the Ultra-Lofty     850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': ' The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection. This is stated in the description of the jacket: "This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable..."'},
 {'query': "What is the material of the Women's Campside Oxfords and what is its special feature?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material, which provides a broken-in feel and look. One special feature of these shoes is the Cleansport NXT® antimicrobial odor control i

In [200]:
from langchain.evaluation.qa import QAEvalChain

In [201]:
eval_chain = QAEvalChain.from_llm(llm_nvidia)

In [202]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [204]:
graded_outputs

[{'results': ' CORRECT\n\nThe student answer is factually correct. The Cozy Comfort Pullover Set does have side pockets, as stated in the "Additional Features" section of the set\'s description. The student\'s answer provides additional context, but it does not contain any conflicting information.'},
 {'results': ' CORRECT\n\nExplanation:\nThe student answer correctly identifies the Ultra-Lofty 850 Stretch Down Hooded Jacket as being from the DownTek collection. The additional information provided in the student answer does not contradict the true answer, so it is still considered correct.'},
 {'results': " INCORRECT\n\nAlthough the student answer correctly identifies the material of the Women's Campside Oxfords as soft canvas, the special feature mentioned in the student answer (super-soft canvas and thick cushioning) is not the same as the special feature given in the true answer (Cleansport NXT® antimicrobial odor control in the innersole)."},
 {'results': " GRADE: CORRECT\n\nThe st

In [207]:
for i, eg in enumerate(examples):
    print("_"*50)
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print("_"*50)

__________________________________________________
Example 0:
Question: Do the Cozy Comfort Pullover Set    have side pockets?
Real Answer: Yes
Predicted Answer:  Yes, the Cozy Comfort Pullover Set does have side pockets. This information can be found in the "Additional Features" section of the set's description. It mentions that the pull-on pants have "side pockets."
Predicted Grade:  CORRECT

The student answer is factually correct. The Cozy Comfort Pullover Set does have side pockets, as stated in the "Additional Features" section of the set's description. The student's answer provides additional context, but it does not contain any conflicting information.
__________________________________________________
__________________________________________________
Example 1:
Question: What collection is the Ultra-Lofty     850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer:  The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection

# Agents

In [241]:
#!pip install langchain-experimental
# !pip install numexpr
#!pip install wikipedia
#!pip install langchainhub

In [291]:
from langchain.agents import AgentExecutor, load_tools, create_react_agent 
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

In [292]:
tools = load_tools(["llm-math","wikipedia"], llm=llm_nvidia)

In [341]:
prompt_text = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''
prompt = PromptTemplate.from_template(prompt_text)
agent = create_react_agent(llm = llm_nvidia, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [344]:
question = {
    "input": "What is the 25% of 300?"
}

result = agent_executor.invoke(question)
print(result['output'])

The 25% of 300 is 75.0.


## Wikipedia example

In [345]:
question = {
    "input" : "Tom Michael Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
}
result = agent_executor.invoke(question) 
print(result['output'])

Thomas G. Mitchell, a computer scientist from Carnegie Mellon University, wrote a book, but I do not have information about the title or content of the book.


## Python Agent

In [347]:
python_repl = PythonREPL()
python_repl.run("print(2+2)")

'4\n'

In [ ]:
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
agent = create_react_agent(llm = llm_nvidia, tools = [repl_tool], prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=[repl_tool], handle_parsing_errors=True)


In [333]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [339]:
question = {
    "input": f"""Sort these customers by last name 
    and then first name 
    and print the output: {customer_list}"""
}

result = agent_executor.invoke(question)

In [340]:
print(result['output'])

The sorted list of customers by last name and then first name is:
[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]
